In [15]:
import numpy as np
import pandas as pd
from neuromaps.images import load_data, load_gifti, annot_to_gifti, relabel_gifti, construct_shape_gii
from neuromaps.datasets import fetch_annotation
from neuromaps.resampling import resample_images
from neuromaps.nulls import alexander_bloch, burt2020
from neuromaps.parcellate import Parcellater
from scipy.stats import pearsonr
from neuromaps import transforms 
from neuromaps.stats import compare_images
from neuromaps.nulls import hungarian

import warnings 
warnings.filterwarnings("ignore", category=DeprecationWarning)
warnings.filterwarnings("ignore", category=FutureWarning)

In [23]:

# define path
path = '/Users/laurituominen/Documents/Research/Reettis/neuromaps/'

# load in different parcellation files
dk_fsaverage_10k = (path + 'parcellations/atlas-desikankilliany_space-fsaverage_den-10k_hemi-L.label.gii.gz',
                     path + 'parcellations/atlas-desikankilliany_space-fsaverage_den-10k_hemi-R.label.gii.gz')
dk_fsaverage_164k = (path + 'parcellations/atlas-desikankilliany_space-fsaverage_den-164k_hemi-L.aparc-1.annot',
                     path + 'parcellations/atlas-desikankilliany_space-fsaverage_den-164k_hemi-R.aparc-1.annot')
dk_mni = path + 'parcellations/atlas-desikankilliany_space-MNI_res-1mm.nii.gz'

# make sure label IDs are consecutive across hemispheres
dk_fsaverage_10k = relabel_gifti(dk_fsaverage_10k)
dk_fsaverage_164k = annot_to_gifti(dk_fsaverage_164k)  # this does relabel_gift and also converts the annot file to gifti

# make the parcellaters for each space
parcellater_fs10k = Parcellater(dk_fsaverage_10k, 'fsaverage')
parcellater_fs164k = Parcellater(dk_fsaverage_164k, 'fsaverage')
parcellater_mni = Parcellater(dk_mni, 'MNI152')

In [24]:
# download enigma
enigmamap = pd.read_csv(path+'data/ENIGMA_S32_partial_correlation_between_cortical_thickness_and_chlorpromazine_equivalents.csv')
enigmamap.drop([68, 69], inplace=True)  # remove the last two rows
enigma_parc = enigmamap['partial_r'].to_numpy()

# download the regions for MNI152 
rois = pd.read_csv(path+'parcellations/atlas-desikankilliany.csv')
rois = rois[(rois['structure'] == 'cortex')].id.to_numpy()


In [28]:
# get sig-maps 
img_L = load_data(path + 'data/lh.sig.nii')
img_gii_L = construct_shape_gii(img_L)

img_R = load_data(path + 'data/rh.sig.nii')  
img_gii_R = construct_shape_gii(img_R)

#parcellate the turku_map into turku_parc
turku_map = (img_gii_L, img_gii_R)
turku_parc = parcellater_fs164k.fit_transform(turku_map, space='fsaverage', ignore_background_data=True)
np.save(path + 'data/turku_parc.npy', turku_parc)

In [8]:
# get annotations 
annotations = list(fetch_annotation(source=['hcps1200',
                                            'margulies2016',
                                            'raichle',
                                            'ding2010', 
                                            'finnema2016', 
                                            'dubois2015',
                                            'dukart2018',
                                            'gallezot2010',
                                            'gallezot2017',
                                            'hillmer2016',
                                            'jaworska2020',
                                            'kaller2017',
                                            'kantonen2020',
                                            'laurikainen2018',
                                            'normandin2015',
                                            'radnakrishnan2018',
                                            'sandiego2015',
                                            'satterthwaite2014',
                                            'sasaki2012',
                                            'savli2012',
                                            'satterthwaite2014',
                                            'smith2017',
                                            'tuominen',
                                            'mueller2013',
                                            'naganawa2020',
                                            'fazio2016']).keys())

annotations.extend(fetch_annotation(source=['norgaard2021', 'beliveau2017'], space='fsaverage').keys())


In [9]:
# parcellate annotations

# initialize
parcellated = dict([])
for (src, desc, space, den) in annotations:

    annot = fetch_annotation(source=src, desc=desc, space=space, den=den)
    
    if space == 'MNI152':
        parcellater = parcellater_mni
    elif space == 'fsaverage' and den == '164k':
        parcellater = parcellater_fs164k
    elif space == 'fsLR' and den == '164k':
        space = 'fsaverage'
        annot = transforms.fslr_to_fsaverage(annot, target_density='164k')
        parcellater = parcellater_fs164k
    elif space == 'fsLR' and den != '164k':
        # unfortunately for fsLR-4k we are upsampling to fsaverage-10k to parcellate but it should be fine
        space = 'fsaverage'
        annot = transforms.fslr_to_fsaverage(annot, target_density='10k')
        parcellater = parcellater_fs10k

    parcellated[desc] = parcellater.fit_transform(annot, space=space, ignore_background_data=True)
    
    if parcellated[desc].shape == (1,83):
        parcellated[desc] = parcellated[desc][0][rois]
  

/Users/laurituominen/anaconda3/lib/python3.10/site-packages/scipy/ndimage/_measurements.py:802: RuntimeWarning: invalid value encountered in divide
  return sum / numpy.asanyarray(count).astype(numpy.float64)


In [10]:
# get spins 
spins = pd.read_csv(path + 'parcellations/spins_hungarian_aparc+aseg_ctx.csv', header=None)
nspins = spins.values.shape[1]


In [22]:
# Correlations between parcellated annotations & parcellated antipsychotic effects on cortical thickness 

# initialize dictionary to save out later
nulls_enigma = dict([])
corrs_enigma = dict([])

nulls_turku = dict([])
corrs_turku = dict([])

# go over annotations 
for src, desc, space, den in annotations:
    if space == 'MNI152':
        parcellation=dk_mni
        
    elif space == 'fsaverage' and den == '164k':
        parcellation=dk_fsaverage_164k
        
    elif space == 'fsLR' and den == '164k':
        parcellation=dk_fsaverage_164k
        
    elif space == 'fsLR' and den != '164k':
        parcellation=dk_fsaverage_10k
    
    # correlation between annotations and enigma map
    rho_enigma = pearsonr(parcellated[desc], enigma_parc)[0]
    rho_turku = pearsonr(parcellated[desc], turku_parc)[0]

    # get 10k rotations 
    rotated = hungarian(data=parcellated[desc], n_perm=10000, spins=spins, parcellation=parcellation) 
    
    # get null Turku
    n = np.zeros((nspins, ))
    for i in range(nspins):
        n[i] = pearsonr(turku_parc, rotated[:,i])[0]    
    
    # get p-value Turku
    pspin = (1 + sum(abs(n) > abs(rho_turku ))) / (nspins + 1)

    # store, multiply by -1 to make more intuitive  
    corrs_turku[src+'_'+desc] = ( (-1 * rho_turku, pspin ) )
    nulls_turku[src+'_'+desc] = n
    
    # get null enigma
    n = np.zeros((nspins, ))
    for i in range(nspins):
        n[i] = pearsonr(enigma_parc, rotated[:,i])[0]    
    
    # get p-value enigma
    pspin = (1 + sum(abs(n) > abs(rho_enigma ))) / (nspins + 1)

    # store, multiply by -1 to make more intuitive
    corrs_enigma[src+'_'+desc] = ( (-1 * rho_enigma, pspin ) )
    nulls_enigma[src+'_'+desc] = n

# save correlations & nulls 
np.savez(path + 'data/corrs_turku.npz', **corrs_turku)
np.savez(path + 'data/nulls_turku.npz', **nulls_turku)
np.savez(path + 'data/nulls_enigma.npz', **nulls_enigma)
np.savez(path + 'data/corrs_enigma.npz', **corrs_enigma)